<a href="https://colab.research.google.com/github/cateto/python4NLP/blob/main/kobert/load_model_success_ing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
# 변경: 최신 버전으로 설치하면 "Input: must be Tensor, not str" 라는 에러 발생
!pip install transformers==3
!pip install torch

     |████████████████████████████████| 46.9 MB 43 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 27.9 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595736 sha256=401481f908a01cc169721b3ef315b557f383c674167b7f2bf34e5ac00d342cfd
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 35.4 MB/s 
     |████████████████████████████████| 754 kB 30.9 MB/s 
     |████████████████████████████████| 895 kB 49.7 MB/s 
     |████████████████████████████████| 3.0 MB 44.2 MB/s 


In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-ote2wdbn
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-ote2wdbn
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12770 sha256=ecbf111e68670ca74372562951c9c1b0f863d34f198a13566a026a7d305fe9fc
  Stored in directory: /tmp/pip-ephem-wheel-cache-jds1df4f/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [4]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768, # bert의 hidden layer 크기?
                #  num_classes=2,
                 num_classes=9, # 분류 class 크기
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes) #nn.Linear
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [5]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [6]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [7]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [8]:
## Setting parameters
max_len = 128 # max_length 512, but Considering Computing Resource => set 163
batch_size = 16 # GPU CUDA problem
warmup_ratio = 0.1
num_epochs = 4
max_grad_norm = 1
log_interval = 200
learning_rate = 1e-5

In [9]:
import torch
device = torch.device('cpu')
model = torch.load('/content/drive/MyDrive/rsn_nlp_project/20210802model_95.pt', map_location=device)

In [78]:
import pandas as pd
import re

lucy_data = pd.read_csv('/content/drive/MyDrive/rsn_nlp_project/lucy_data0729.csv')

lucy_data['contents'] = lucy_data['contents'].str.replace("\(.*\)|\s-\s.*"," " ,regex=True)
lucy_data['contents'] = lucy_data['contents'].str.replace("\[.*\]|\s-\s.*"," ",regex=True)
lucy_data['contents'] = lucy_data['contents'].str.replace("\<.*\>|\s-\s.*"," ",regex=True)
lucy_data['contents'] = lucy_data['contents'].str.replace("무단전재 및 재배포 금지"," ",regex=True)
lucy_data['contents'] = lucy_data['contents'].str.replace("무단 전재 및 재배포 금지"," ",regex=True)
lucy_data['contents'] = lucy_data['contents'].str.replace("©"," ",regex=True)
lucy_data['contents'] = lucy_data['contents'].str.replace("ⓒ"," ",regex=True)
lucy_data['contents'] = lucy_data['contents'].str.replace("저작권자"," ",regex=True)
lucy_data['contents'] = lucy_data['contents'].str.replace(".* 기자", " ", regex=True) #기자 이름에서 오는 유사도 차단
lucy_data['contents'] = lucy_data['contents'].str.replace("사진 = .*", " ", regex=True) #사진 첨부 문구 삭제
lucy_data['contents'] = lucy_data['contents'].str.replace("사진=.*", " ", regex=True) #사진 첨부 문구 삭제
lucy_data['contents'] = lucy_data['contents'].str.replace("([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+.[a-zA-Z0-9-.]+)", " ", regex=True) #이메일 주소에서 오는 유사도 차단
lucy_data['contents'] = lucy_data['contents'].str.replace("\n"," ")
lucy_data['contents'] = lucy_data['contents'].str.replace("\r"," ")
lucy_data['contents'] = lucy_data['contents'].str.replace("\t"," ")
lucy_data['contents'] = lucy_data['contents'].str.replace( "\’" , "", regex=True)
lucy_data['contents'] = lucy_data['contents'].str.replace('\"', "",regex=True)
lucy_data['contents'] = lucy_data['contents'].str.replace("[ ]{2,}"," ",regex=True)

lucy_data[:3]

,id,contents,category
0,DN000000008778200425,- 공정소득은 코로나 이후 악화되는 K자형 양극화에 대비하자는 것 반박 야권의 대선...,정치
1,DN000000008808263362,"- 공동대표 및 정책자문위원 등 임명장 수여 '내 삶을 지켜주는 나라, 신복지 충남...",정치
2,DN000000008791388703,"-- 바르셀로나 MWC21 참가 -- 인텔, 기가테라 커뮤니케이션즈 를 오픈랜 생태...",IT/과학


In [44]:
type(lucy_data['contents'])

pandas.core.series.Series

In [79]:
label_dict = {'0': 'IT/과학',
 '1': '경제',
 '2': '문화',
 '3': '미용/건강',
 '4': '사회',
 '5': '생활',
 '6': '스포츠',
 '7': '연예',
 '8': '정치'}

for key, value in label_dict.items():
  lucy_data['category'] = lucy_data['category'].str.replace(value, key)

lucy_data.to_csv('/content/drive/MyDrive/rsn_nlp_project/lucy_data0804.txt', sep = '\t' , index = False)

In [80]:
lucy_data[:5]

,id,contents,category
0,DN000000008778200425,- 공정소득은 코로나 이후 악화되는 K자형 양극화에 대비하자는 것 반박 야권의 대선...,8
1,DN000000008808263362,"- 공동대표 및 정책자문위원 등 임명장 수여 '내 삶을 지켜주는 나라, 신복지 충남...",8
2,DN000000008791388703,"-- 바르셀로나 MWC21 참가 -- 인텔, 기가테라 커뮤니케이션즈 를 오픈랜 생태...",0
3,DN000000008853835944,더불어민주당 신영대 의원 이 민주당의 혁신과 흥행을 담당할 대선경선기획단...,8
4,DN000000008834209192,지난 19일 미·멕시코 국경지대인 텍사스주 엘패소와 멕시코 시우대드후아레스 사이에...,2


{0: 'IT/과학',
 1: '경제',
 2: '문화',
 3: '미용/건강',
 4: '사회',
 5: '생활',
 6: '스포츠',
 7: '연예',
 8: '정치'}

In [90]:
test_set = nlp.data.TSVDataset("/content/drive/MyDrive/rsn_nlp_project/lucy_data0804.txt", field_indices=[1,2], num_discard_samples=1)

In [86]:
test_set[22:23]

[[' 우리들의공부방 지역아동센터에서 응원하고 있는 아이들 지난 5월 26일, 관악구 지역아동센터 어린이, 청소년을 대상으로 겨자씨 영어말하기 대회가 열렸다. 겨자씨어머니봉사회 의 기획으로 시작된 영어말하기 대회는 올해로 11회째를 맞이했다. 행사는 코로나19로 인해 대회 사상 처음으로 온라인으로 유튜브와 화상대화 프로그램 ZOOM을 활용해 진행됐다. 약 2시간 동안 이어진 대회는 1부 개회식, 2부 영어스피킹 대회, 3부 공연으로 진행됐다. 어린이, 청소년 행사인 만큼 영어말하기 대회 참가자뿐만 아니라, 관악구 지역아동센터 27개소의 청소년이 참여할 수 있는 선물 이벤트를 마련해 진행했다. 행사 관계자는 “대회가 단순히 우열을 가리는 게 아니라, 응원하는 아이들도 함께 즐거운 축제가 되기를 바라는 마음으로 선물이라는 장치를 곳곳에 마련했다. 2시간 내내 퀴즈나 선물 뽑기 과정을 통해 27개 센터에 균등하게 돌아갈 수 있는 선물이벤트를 마련해 다른 즐거움을 주려고 노력했다.”라고 밝혔다. 줌으로 행사에 참여하고 있는 아이들의 모습 이벤트 결과를 발표하고 있는 모습. 왼쪽부터 사회자 황정민, 오은영 회장, 김선희 센터장. 발표는 총 18팀이 영어 스피치로 진행됐다. 주제는 코로나19, 가족, 좋아하는 음식, 게임 소개 등 다양했다. 참관한 아이들은 각 센터에서 ZOOM을 통해 다른 친구들의 발표를 보며 박수와 응원을 보내는 등 적극적인 참여가 이뤄졌다. 영어말하기 대회 행사사진1 공연 사진. 위에서부터 관악구 솔까치 합창단, 비전교실 지역아동센터 예나은하 팀, 해성지역아동센터 해성소년단 댄스팀 발표 후엔 어린이, 청소년의 공연이 이어졌다. 관악구 솔까치 합창단의 노래 를 볼 수 있었다. 참좋은지역아동센터에서 응원하고 있는 아이들의 모습 발표 중간마다 이벤트는 희망신나는집문화학교 김선희 센터장이 진행했다. 아이들은 퀴즈와 제비뽑기 이벤트에 카카오톡 채팅창과 행사 유튜브 댓글, 센터별 선생님의 도움을 통해 참여했다. 그 결과 약 70명의 친구들이 선물과 상금을 받았

In [91]:
test_set = BERTDataset(test_set, 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=9)

predict_list = []

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = model(token_ids, valid_length, segment_ids)
  print(out)
  predict_list.append(out.max(dim=0)[1])



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 9 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


tensor([[-0.8502, -0.4415, -0.5896, -0.7716, -0.0983, -1.4358, -0.4181, -0.8327,
          6.1123]], grad_fn=<AddmmBackward>)
tensor([[-0.9807, -0.6544, -0.6004, -0.8391,  0.2900, -1.4280, -0.3681, -0.9093,
          6.1523]], grad_fn=<AddmmBackward>)
tensor([[ 6.1042,  0.0087, -0.8891, -0.5300, -1.4322, -0.6924, -1.0116, -0.6454,
         -0.8131]], grad_fn=<AddmmBackward>)
tensor([[-0.8007, -0.4285, -0.6174, -0.6729, -0.2679, -1.5598, -0.3396, -0.6945,
          6.1096]], grad_fn=<AddmmBackward>)
tensor([[-1.1142, -1.1549, -0.6022, -1.2238,  6.4595,  0.2228, -0.8804, -1.2948,
         -0.2356]], grad_fn=<AddmmBackward>)
tensor([[-1.3082, -0.7569, -0.9347, -0.6588,  6.2511, -0.2711, -0.5561, -1.2225,
         -0.3617]], grad_fn=<AddmmBackward>)
tensor([[ 0.5278, -1.2435, -1.0398, -1.1404,  5.9565, -0.4664, -1.2786, -1.5387,
          0.3019]], grad_fn=<AddmmBackward>)


KeyboardInterrupt: ignored